In [49]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.model_selection import train_test_split
import pandas as pd
from numpy import genfromtxt
from sklearn.decomposition import PCA
import numpy as np
import csv
import matplotlib.pyplot as plt

In [2]:
training_data = genfromtxt('training.csv',dtype = float, delimiter=',')
training_data = training_data[1:, 1:32]

# Separating labels for training

In [3]:
training_labels = genfromtxt('training.csv',dtype = str, delimiter=',')
training_labels = training_labels[1:,-1]
training_labels = np.reshape(training_labels,(len(training_labels),1))

In [4]:
a, b = np.where(training_data == -999)
new_r, new_c = np.where(training_data != -999)
mean_list = np.mean(training_data[new_r][new_c], axis = 0)

In [5]:
for i in range(len(a)):
    training_data[a[i]][b[i]] = mean_list[b[i]]

x, y = np.where(training_data == -999)
print(x,y)
np.savetxt("pca_higgs_boson_data.csv", training_data, delimiter=",")

[] []


In [7]:
scaler = StandardScaler()
# Fit on training set only
scaler.fit(training_data[2:])
# print(training_data[1])

StandardScaler(copy=True, with_mean=True, with_std=True)

In [8]:
training_data = scaler.transform(training_data)

In [9]:
pca = PCA(.95)

In [10]:
pca.fit(training_data)

PCA(copy=True, iterated_power='auto', n_components=0.95, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

In [13]:
pca.n_components_
pca.explained_variance_

array([8.38122291, 2.86242727, 2.45472189, 1.93739124, 1.62457038,
       1.47333865, 1.30247761, 1.19896637, 1.07527104, 1.01889864,
       0.92345176, 0.87886605, 0.77987371, 0.74635684, 0.61387006,
       0.53136428, 0.48549127, 0.44548344, 0.44031435, 0.41417013])

In [14]:
training_data = pca.transform(training_data)

In [15]:
np.savetxt("Check_pca_higgs_boson_data.csv", training_data, delimiter=",")

In [21]:
training_labels = (training_labels == 's').astype(np.float32)

In [25]:
train = np.concatenate((training_data,training_labels),axis=1)

# Trying XGBoost

In [26]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [27]:
X,Y = train[:,:-1],train[:,-1]
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [44]:
modelXGB = xgb.XGBClassifier(learning_rate = 0.2,max_depth=9)
modelXGB = model.fit(X_train, Y_train)

In [41]:
Y_predicted = model.predict(X_test)

In [46]:
accuracy = accuracy_score(Y_test, Y_predicted)
print(accuracy*100)

94.372


# Decision Tree Classifier

In [55]:
from sklearn import tree

In [69]:
modelDT = tree.DecisionTreeClassifier(max_depth= 30, criterion='entropy')

In [70]:
modelDT.fit(X_train, Y_train)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=15,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [71]:
Y_predicted = modelDT.predict(X_test)

In [72]:
accuracy = accuracy_score(Y_test, Y_predicted)
print(accuracy*100)

89.004
